# Multiple Images Validation

In [1]:
from ultralytics import YOLO
from PIL import Image
from typing import List, Tuple
import os

In [2]:
# Cargar el modelo
model = YOLO('../data/models/500epochs/v11/last.pt')

# Directorios de imágenes y etiquetas
image_dir = '../data/testing_data/images'

label_dir = '../data/testing_data/labels'


In [3]:
# Función para calcular IoU
def iou(boxA: Tuple[float, float, float, float], boxB: Tuple[float, float, float, float]) -> float:
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[0] + boxA[2], boxB[0] + boxB[2])
    yB = min(boxA[1] + boxA[3], boxB[1] + boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = boxA[2] * boxA[3]
    boxBArea = boxB[2] * boxB[3]
    unionArea = boxAArea + boxBArea - interArea
    return interArea / unionArea if unionArea != 0 else 0

In [4]:
# Función para evaluar las métricas por categoría
def evaluate(true_boxes: List[Tuple[float, float, float, float]], pred_boxes: List[Tuple[float, float, float, float]], iou_threshold: float = 0.5) -> Tuple[int, int, int]:
    tp, fp, fn = 0, 0, 0
    matched_pred_indices = set()
    for true_box in true_boxes:
        match_found = False
        for i, pred_box in enumerate(pred_boxes):
            if i in matched_pred_indices:
                continue
            if iou(true_box, pred_box) >= iou_threshold:
                tp += 1
                matched_pred_indices.add(i)
                match_found = True
                break
        if not match_found:
            fn += 1
    fp = len(pred_boxes) - len(matched_pred_indices)
    return tp, fp, fn

In [5]:
# Inicialización de contadores globales para cada categoría
global_tp = {"ball": 0, "goalkeeper": 0, "player": 0, "referee": 0}
global_fp = {"ball": 0, "goalkeeper": 0, "player": 0, "referee": 0}
global_fn = {"ball": 0, "goalkeeper": 0, "player": 0, "referee": 0}

# Procesar cada imagen y sus etiquetas
for image_file in os.listdir(image_dir):
    if not image_file.endswith(".jpg"):
        continue
    image_path = os.path.join(image_dir, image_file)
    label_path = os.path.join(label_dir, image_file.replace(".jpg", ".txt"))

    image = Image.open(image_path)
    img_width, img_height = image.size

    # Leer las cajas de verdad (true boxes)
    true_boxes_ball, true_boxes_goalkeeper, true_boxes_player, true_boxes_referee = [], [], [], []
    with open(label_path, 'r') as file:
        for line in file:
            label, x_center, y_center, width, height = map(float, line.split())
            bbox = (x_center * img_width, y_center * img_height, width * img_width, height * img_height)
            if label == 0:
                true_boxes_ball.append(bbox)
            elif label == 1:
                true_boxes_goalkeeper.append(bbox)
            elif label == 2:
                true_boxes_player.append(bbox)
            elif label == 3:
                true_boxes_referee.append(bbox)

    # Realizar detección con el modelo
    results = model(image_path)
    predicted_boxes_ball, predicted_boxes_goalkeeper, predicted_boxes_player, predicted_boxes_referee = [], [], [], []
    for box in results[0].boxes:
        label = box.cls.item()
        bbox = (box.xywh[0][0].item(), box.xywh[0][1].item(), box.xywh[0][2].item(), box.xywh[0][3].item())
        if label == 0:
            predicted_boxes_ball.append(bbox)
        elif label == 1:
            predicted_boxes_goalkeeper.append(bbox)
        elif label == 2:
            predicted_boxes_player.append(bbox)
        elif label == 3:
            predicted_boxes_referee.append(bbox)

    # Calcular métricas para cada categoría
    for category, true_boxes, pred_boxes in zip(
        ["ball", "goalkeeper", "player", "referee"],
        [true_boxes_ball, true_boxes_goalkeeper, true_boxes_player, true_boxes_referee],
        [predicted_boxes_ball, predicted_boxes_goalkeeper, predicted_boxes_player, predicted_boxes_referee]
    ):
        tp, fp, fn = evaluate(true_boxes, pred_boxes)
        global_tp[category] += tp
        global_fp[category] += fp
        global_fn[category] += fn

# Cálculo de precisión y recall global por categoría
for category in global_tp.keys():
    precision = global_tp[category] / (global_tp[category] + global_fp[category]) if (global_tp[category] + global_fp[category]) > 0 else 0
    recall = global_tp[category] / (global_tp[category] + global_fn[category]) if (global_tp[category] + global_fn[category]) > 0 else 0
    print(f"{category.capitalize()} Precision: {precision:.2f}")
    print(f"{category.capitalize()} Recall: {recall:.2f}")


image 1/1 C:\Users\mique\Desktop\Master Ciencia de Dades UOC\Treball de Final de Master\football_analysis_system\development_and_analysis\..\data\testing_data\images\08fd33_3_1_png.rf.894a7edd6b726ed886b3777ea55d6beb.jpg: 640x640 19 players, 2 referees, 868.8ms
Speed: 3.0ms preprocess, 868.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Users\mique\Desktop\Master Ciencia de Dades UOC\Treball de Final de Master\football_analysis_system\development_and_analysis\..\data\testing_data\images\08fd33_3_3_png.rf.7aaf5d38e77c064882cb7b0f80e7e156.jpg: 640x640 1 ball, 20 players, 2 referees, 878.8ms
Speed: 4.5ms preprocess, 878.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Users\mique\Desktop\Master Ciencia de Dades UOC\Treball de Final de Master\football_analysis_system\development_and_analysis\..\data\testing_data\images\08fd33_9_8_png.rf.c4792b7740c5e581674385c8a2c149fe.jpg: 640x640 20 players, 2 referees, 847.6ms
Speed: 

# Single Image Validation

In [12]:
from ultralytics import YOLO
from PIL import Image
from typing import List, Tuple

In [13]:
# Cargar el modelo
model = YOLO('../data/models/v8/best.pt')

# Rutas de la imagen y etiquetas
image_path = '../data/testing_data/images/2e57b9_1_6_png.rf.d75ca2d49fa77ff7b77446b62f9dbe6d.jpg'
image = Image.open(image_path)
img_width, img_height = image.size
label_path = '../data/testing_data/labels/2e57b9_1_6_png.rf.d75ca2d49fa77ff7b77446b62f9dbe6d.txt'

In [25]:
true_boxes_ball = []
true_boxes_goalkeeper = []
true_boxes_player = []
true_boxes_referee = []

with open(label_path, 'r') as file:
    for line in file:
        label, x_center, y_center, width, height = map(float, line.split())
        if label == 0:
            true_boxes_ball.append((x_center, y_center, width, height))
        elif label == 1:
            true_boxes_goalkeeper.append((x_center, y_center, width, height))
        elif label == 2:
            true_boxes_player.append((x_center, y_center, width, height))
        elif label == 3:
            true_boxes_referee.append((x_center, y_center, width, height))

true_boxes_ball_px = [(x * img_width, y * img_height, w * img_width, h * img_height) for x, y, w, h in true_boxes_ball]
true_boxes_goalkeeper_px = [(x * img_width, y * img_height, w * img_width, h * img_height) for x, y, w, h in true_boxes_goalkeeper]
true_boxes_player_px = [(x * img_width, y * img_height, w * img_width, h * img_height) for x, y, w, h in true_boxes_player]
true_boxes_referee_px = [(x * img_width, y * img_height, w * img_width, h * img_height) for x, y, w, h in true_boxes_referee]


print("True Boxes Ball:", true_boxes_ball_px)
print("True Boxes Goalkeeper:", true_boxes_goalkeeper_px)
print("True Boxes Player:", true_boxes_player_px)
print("True Boxes Referee:", true_boxes_referee_px)

True Boxes Ball: [(763.5, 623.5, 9.5, 17.5)]
True Boxes Goalkeeper: [(995.0, 479.5, 14.5, 54.0)]
True Boxes Player: [(655.5, 479.5, 11.5, 41.5), (583.5, 499.0, 11.5, 50.0), (940.0, 589.0, 14.5, 61.5), (932.5, 553.5, 13.5, 59.5), (846.0, 536.5, 12.0, 55.5), (962.0, 580.5, 14.5, 59.5), (353.5, 562.0, 12.5, 59.5), (826.0, 521.5, 12.0, 54.5), (722.0, 525.0, 13.5, 52.0), (294.5, 559.5, 10.0, 59.5), (475.0, 619.5, 14.0, 67.5), (737.5, 620.0, 16.0, 61.5), (531.5, 607.0, 12.0, 64.0), (716.5, 619.5, 16.0, 63.0), (688.0, 608.0, 22.5, 54.5), (844.0, 672.5, 14.5, 70.0), (244.0, 737.0, 28.0, 80.5), (342.0, 777.0, 20.0, 91.5), (501.5, 742.0, 24.0, 83.0), (958.5, 558.0, 13.5, 54.5)]
True Boxes Referee: [(624.5, 539.5, 10.5, 47.5), (1209.0, 717.5, 28.5, 74.0), (235.5, 423.5, 13.5, 42.5)]


In [16]:
# Realizar detecciones

results = model(image_path)
detected_boxes = [(box.cls.item(), box.xywhn.tolist()) for box in results[0].boxes]
detected_boxes_px = [(box.cls.item(), box.xywh.tolist()) for box in results[0].boxes]

predicted_boxes_ball = []
predicted_boxes_goalkeeper = []
predicted_boxes_player = []
predicted_boxes_referee = []

for box in detected_boxes:
    label = box[0]
    bbox = (box[1][0][0], box[1][0][1], box[1][0][2], box[1][0][3])
    if label == 0:
        predicted_boxes_ball.append(bbox)
    elif label == 1:
        predicted_boxes_goalkeeper.append(bbox)
    elif label == 2:
        predicted_boxes_player.append(bbox)
    elif label == 3:
        predicted_boxes_referee.append(bbox)

predicted_boxes_ball_px = []
predicted_boxes_goalkeeper_px = []
predicted_boxes_player_px = []
predicted_boxes_referee_px = []

for box in detected_boxes_px:
    label = box[0]
    bbox = (box[1][0][0], box[1][0][1], box[1][0][2], box[1][0][3])
    if label == 0:
        predicted_boxes_ball_px.append(bbox)
    elif label == 1:
        predicted_boxes_goalkeeper_px.append(bbox)
    elif label == 2:
        predicted_boxes_player_px.append(bbox)
    elif label == 3:
        predicted_boxes_referee_px.append(bbox)

print("Predicted Boxes Ball:", predicted_boxes_ball_px)
print("Predicted Boxes Goalkeeper:", predicted_boxes_goalkeeper_px)
print("Predicted Boxes Player:", predicted_boxes_player_px)
print("Predicted Boxes Referee:", predicted_boxes_referee_px)



image 1/1 C:\Users\mique\Desktop\Master Ciencia de Dades UOC\Treball de Final de Master\football_analysis_system\development_and_analysis\..\data\testing_data\images\2e57b9_1_6_png.rf.d75ca2d49fa77ff7b77446b62f9dbe6d.jpg: 640x640 1 goalkeeper, 20 players, 3 referees, 62.5ms
Speed: 15.6ms preprocess, 62.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
Predicted Boxes Ball: []
Predicted Boxes Goalkeeper: [(181.0667724609375, 324.4131774902344, 21.13983154296875, 56.32000732421875)]
Predicted Boxes Player: [(845.052001953125, 733.1133422851562, 23.2237548828125, 87.5994873046875), (498.93133544921875, 483.0350646972656, 21.6455078125, 59.49993896484375), (998.9293212890625, 604.919677734375, 17.203857421875, 83.786376953125), (576.8115234375, 812.5552978515625, 35.0401611328125, 85.099853515625), (559.6214599609375, 307.0665588378906, 10.283447265625, 54.3575439453125), (122.874755859375, 560.012451171875, 21.515472412109375, 76.1536865234375), (524.6072387695312, 349

In [17]:
def iou(boxA: Tuple[float, float, float, float], boxB: Tuple[float, float, float, float]) -> float:
    # Calcular coordenadas de la intersección
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[0] + boxA[2], boxB[0] + boxB[2])
    yB = min(boxA[1] + boxA[3], boxB[1] + boxB[3])

    # Área de la intersección
    interArea = max(0, xB - xA) * max(0, yB - yA)
    # Áreas de cada caja
    boxAArea = boxA[2] * boxA[3]
    boxBArea = boxB[2] * boxB[3]

    # Área de la unión
    unionArea = boxAArea + boxBArea - interArea

    # IoU
    return interArea / unionArea if unionArea != 0 else 0

In [18]:
def evaluate(true_boxes: List[Tuple[float, float, float, float]],
             pred_boxes: List[Tuple[float, float, float, float]],
             iou_threshold: float = 0.5) -> Tuple[float, float]:
    
    tp, fp, fn = 0, 0, 0
    matched_pred_indices = set()

    for true_box in true_boxes:
        match_found = False
        for i, pred_box in enumerate(pred_boxes):
            if i in matched_pred_indices:
                continue
            iou_value = iou(true_box, pred_box)
            if iou_value >= iou_threshold:
                tp += 1
                matched_pred_indices.add(i)
                match_found = True
                break
        if not match_found:
            fn += 1

    fp = len(pred_boxes) - len(matched_pred_indices)

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0

    return precision, recall

In [19]:
ball_precision, ball_recall = evaluate(true_boxes_ball_px, predicted_boxes_ball_px)
goalkeeper_precision, goalkeeper_recall = evaluate(true_boxes_goalkeeper_px, predicted_boxes_goalkeeper_px)
player_precision, player_recall = evaluate(true_boxes_player_px, predicted_boxes_player_px)
referee_precision, referee_recall = evaluate(true_boxes_referee_px, predicted_boxes_referee_px)

print("Ball Precision:", ball_precision)
print("Ball Recall:", ball_recall)
print("Goalkeeper Precision:", goalkeeper_precision)
print("Goalkeeper Recall:", goalkeeper_recall)
print("Player Precision:", player_precision)
print("Player Recall:", player_recall)
print("Referee Precision:", referee_precision)
print("Referee Recall:", referee_recall)

Ball Precision: 0
Ball Recall: 0.0
Goalkeeper Precision: 1.0
Goalkeeper Recall: 1.0
Player Precision: 1.0
Player Recall: 1.0
Referee Precision: 1.0
Referee Recall: 1.0
